### PENDIENTE
### Cambiar csv por el enlace 
### Comentar cada paso (argumentando lo que hacemos)
### Precio total o precio por metro cuadrado - cual es el mejor para el proyecto?

In [370]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, train_test_split


In [371]:
# Cargar el archivo CSV
df = pd.read_csv('pisos.csv')

# Mostrar las primeras filas para inspeccionar el dataset
df.head()

C:\Users\Usuario\AppData\Local\Temp\ipykernel_204172\402079931.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pisos.csv')


,Unnamed: 0,summary,description,location,photo,recomendado,price,size,rooms,price/m2,bathrooms,Num Photos,type,region
0,0,Casa en calle Urb. Las Mimosas,NaN,Ames (San Tome),https://fotos4.imghs.net/nrd/1069/104/1069_I00...,Recomendado,250.000 €,315 m²,4,793 €/m²,3,31,pisos,a_coruna
1,1,Piso en Milladoiro,NaN,Ames (San Tome),https://fotos4.imghs.net/nrd/1069/113/1069_I00...,Recomendado,90.000 €,72 m²,2,1.250 €/m²,1,11,pisos,a_coruna
2,2,Casa en calle La Piedra,NaN,A Ortigueira (Cariño),https://fotos1.imghs.net/nrd//999170/892070610...,Recomendado,50.000 €,92 m²,2,543 €/m²,1,40,pisos,a_coruna
3,3,Casa en Perillo,NaN,Perillo (Oleiros),https://fotos3.imghs.net/nrd/1089/198/1089_XA2...,Recomendado,359.000 €,244 m²,4,1.471 €/m²,3,39,pisos,a_coruna
4,4,Chalet en Urbanización Las Mimosas,NaN,O Milladoiro (Ames),https://fotos5.imghs.net/nrd/1097/754/1097_0_f...,Recomendado,200.430 €,315 m²,4,636 €/m²,3,24,pisos,a_coruna


In [372]:
# Mostrar información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777179 entries, 0 to 777178
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Unnamed: 0   777179 non-null  int64 
 1   summary      777179 non-null  object
 2   description  760349 non-null  object
 3   location     776886 non-null  object
 4   photo        776305 non-null  object
 5   recomendado  28241 non-null   object
 6   price        776898 non-null  object
 7   size         776892 non-null  object
 8   rooms        752352 non-null  object
 9   price/m2     744608 non-null  object
 10  bathrooms    752526 non-null  object
 11  Num Photos   776898 non-null  object
 12  type         776898 non-null  object
 13  region       776282 non-null  object
dtypes: int64(1), object(13)
memory usage: 83.0+ MB


In [373]:
# Eliminar columnas innecesarias
df.drop(columns=['Unnamed: 0', 'description', 'photo', 'Num Photos', 'recomendado'], inplace=True)

# Quitar símbolos "€ y m²" en 'price' y 'size' y convertir a numérico
df['price'] = pd.to_numeric(df['price'].str.replace('€', '').str.replace(',', ''), errors='coerce')
df['size'] = pd.to_numeric(df['size'].str.replace('m²', '').str.replace(',', ''), errors='coerce')

# Inspeccionar cambios
df[['price', 'size']].head()


,price,size
0,250.00,315.0
1,90.00,72.0
2,50.00,92.0
3,359.00,244.0
4,200.43,315.0


In [374]:
# Eliminar filas con valores nulos
df = df.dropna(subset=['price', 'size', 'rooms', 'bathrooms', 'location', 'region', 'price/m2'])

# Verificar si todavía quedan valores faltantes
print(df.isnull().sum())

summary      0
location     0
price        0
size         0
rooms        0
price/m2     0
bathrooms    0
type         0
region       0
dtype: int64


In [375]:
# Convertir las columnas 'rooms' y 'bathrooms' a formato numérico
df['rooms'] = pd.to_numeric(df['rooms'], errors='coerce')
df['bathrooms'] = pd.to_numeric(df['bathrooms'], errors='coerce')
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['size'] = pd.to_numeric(df['size'], errors='coerce')

In [376]:
# Obtener valores únicos en la columna 'type'
unique_types = df['type'].unique()

# Imprimir los valores únicos
print("Valores únicos en 'type':")
print(unique_types)

Valores únicos en 'type':
['pisos' 'casas' 'piso' 'aticos' 'duplexs' 'estudios' 'fincas_rusticas'
 'lofts']


In [377]:
# Reemplazar 'piso' por 'pisos' en la columna 'type'
df['type'] = df['type'].replace('piso', 'pisos')

# Verificar los cambios
print(df['type'].value_counts())


type
pisos              588551
casas               85984
duplexs             12107
aticos              10536
fincas_rusticas      4179
estudios              523
lofts                 401
Name: count, dtype: int64


In [378]:
# Crear términos de interacción
df['size_rooms_interaction'] = df['size'] * df['rooms']
df['size_bathrooms_interaction'] = df['size'] * df['bathrooms']
df['rooms_bathrooms_interaction'] = df['rooms'] * df['bathrooms']

In [379]:
# Calcular cuartiles
Q1 = df['size'].quantile(0.25)
Q3 = df['size'].quantile(0.75)
IQR = Q3 - Q1

# Definir límites para detectar valores atípicos
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar los valores atípicos
outliers = df[(df['size'] < lower_bound) | (df['size'] > upper_bound)]

print(f"Número de valores atípicos en 'size': {outliers.shape[0]}")
print(outliers[['size']])

Número de valores atípicos en 'size': 58858
         size
0       315.0
4       315.0
8       300.0
12      292.0
14      348.0
...       ...
777121  525.0
777162  461.0
777164  445.0
777168  460.0
777169  293.0

[58858 rows x 1 columns]


In [380]:
# Calcular cuartiles para la columna 'rooms'
Q1_rooms = df['rooms'].quantile(0.25)
Q3_rooms = df['rooms'].quantile(0.75)
IQR_rooms = Q3_rooms - Q1_rooms

# Definir límites para detectar valores atípicos
lower_bound_rooms = Q1_rooms - 1.5 * IQR_rooms
upper_bound_rooms = Q3_rooms + 1.5 * IQR_rooms

# Filtrar los valores atípicos
outliers_rooms = df[(df['rooms'] < lower_bound_rooms) | (df['rooms'] > upper_bound_rooms)]

print(f"Número de valores atípicos en 'rooms': {outliers_rooms.shape[0]}")
print(outliers_rooms[['rooms']].head(10))  # Mostrar los primeros 10 valores atípicos


Número de valores atípicos en 'rooms': 6680
      rooms
14        9
247       8
275       8
296       9
389      12
613       8
731       8
741       8
989       9
1106      8


In [381]:
# Calcular cuartiles para la columna 'bathrooms'
Q1_bathrooms = df['bathrooms'].quantile(0.25)
Q3_bathrooms = df['bathrooms'].quantile(0.75)
IQR_bathrooms = Q3_bathrooms - Q1_bathrooms

# Definir límites para detectar valores atípicos
lower_bound_bathrooms = Q1_bathrooms - 1.5 * IQR_bathrooms
upper_bound_bathrooms = Q3_bathrooms + 1.5 * IQR_bathrooms

# Filtrar los valores atípicos
outliers_bathrooms = df[(df['bathrooms'] < lower_bound_bathrooms) | (df['bathrooms'] > upper_bound_bathrooms)]

print(f"Número de valores atípicos en 'bathrooms': {outliers_bathrooms.shape[0]}")
print(outliers_bathrooms[['bathrooms']].head(10))  # Mostrar los primeros 10 valores atípicos


Número de valores atípicos en 'bathrooms': 35579
     bathrooms
8          4.0
19         4.0
42         4.0
45         4.0
51         5.0
52         4.0
70         4.0
115        4.0
138        5.0
144        5.0


In [382]:
# Filtrar el DataFrame para eliminar los valores atípicos en 'size', 'rooms', y 'bathrooms'
df_cleaned_final = df[
    (df['size'] >= lower_bound) & (df['size'] <= upper_bound) &
    (df['rooms'] >= lower_bound_rooms) & (df['rooms'] <= upper_bound_rooms) &
    (df['bathrooms'] >= lower_bound_bathrooms) & (df['bathrooms'] <= upper_bound_bathrooms)
]


### Perform Train Test Split

In [383]:
# Definir características (features) y variable objetivo (target)
X = df_cleaned_final[['size', 'rooms', 'bathrooms', 'type', 'region', 
                       'size_rooms_interaction', 'size_bathrooms_interaction', 'rooms_bathrooms_interaction']]
y = df_cleaned_final['price']  # La variable objetivo

# Preprocesar características categóricas y escalar numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['size', 'rooms', 'bathrooms', 
                                    'size_rooms_interaction', 
                                    'size_bathrooms_interaction', 
                                    'rooms_bathrooms_interaction']),  
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['type', 'region'])  
    ],
    remainder='passthrough'  
)

# Crear un pipeline que primero preprocesa y luego ajusta el modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE): {mse:.2f}")
print(f"R² score del modelo en el conjunto de prueba: {r2:.4f}")


Error cuadrático medio (MSE): 7989.33
R² score del modelo en el conjunto de prueba: 0.5354


### Model Selection / In this exercise we will be using KNN as our predictive model.

In [ ]:
# Definir características (features) y variable objetivo (target)
X = df_cleaned_final[['size', 'rooms', 'bathrooms', 'type', 'region', 
                       'size_rooms_interaction', 'size_bathrooms_interaction', 'rooms_bathrooms_interaction']]
y = df_cleaned_final['price']  # La variable objetivo

# Preprocesar características categóricas y escalar numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['size', 'rooms', 'bathrooms', 
                                    'size_rooms_interaction', 
                                    'size_bathrooms_interaction', 
                                    'rooms_bathrooms_interaction']),  
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['type', 'region'])  
    ],
    remainder='passthrough'  
)

# Crear el modelo KNN
knn_model = KNeighborsRegressor()

# Crear un pipeline que primero preprocesa y luego ajusta el modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', knn_model)
])

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Ajustar el modelo a los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Error cuadrático medio (MSE) con KNN: {mse:.2f}")
print(f"R² score del modelo en el conjunto de prueba: {r2:.4f}")

# Optimización de hiperparámetros con GridSearchCV
param_grid = {
    'regressor__n_neighbors': [3, 5, 7, 10, 15],  # Número de vecinos
    'regressor__weights': ['uniform', 'distance'],  # Peso de los vecinos
}

# Configurar la búsqueda de hiperparámetros
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, verbose=2)

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Obtener el mejor modelo
best_knn_model = grid_search.best_estimator_

# Realizar predicciones con el mejor modelo
y_pred_best = best_knn_model.predict(X_test)

# Evaluar el rendimiento del mejor modelo
mse_best = mean_squared_error(y_test, y_pred_best)
r2_best = r2_score(y_test, y_pred_best)

print(f"Error cuadrático medio (MSE) con KNN optimizado: {mse_best:.2f}")
print(f"R² score del modelo en el conjunto de prueba: {r2_best:.4f}")
